In [19]:
%reload_ext autoreload
%autoreload 2

import sys
sys.path.append("../../")

import os
print(os.getcwd())

import tempfile
from pathlib import Path
import numpy as np
from torch.utils.data import DataLoader
from models.basic import CNN, BidirectionalRNN
from torch import nn
from loss_module.stroke_recovery_loss import StrokeLoss
import torch
from models.CoordConv import CoordConv
from trainers import TrainerStrokeRecovery
from hwr_utils.stroke_dataset import StrokeRecoveryDataset, read_img
from hwr_utils.stroke_recovery import *
from hwr_utils import utils, stroke_recovery
from torch.optim import lr_scheduler
from timeit import default_timer as timer
from train_stroke_recovery import graph
from hwr_utils.distortions import *
from hwr_utils.stroke_plotting import *


## DRAW THE IMAGE
#Pillow-simd

from PIL import Image, ImageDraw

def loader():
    folder = Path("online_coordinate_data/3_stroke_vSmall")
    folder = Path("online_coordinate_data/8_stroke_vSmall_16")
    folder = Path("online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull")

    print(os.getcwd())
    x_relative_positions = True
    test_size = 5
    train_size = None
    batch_size=32

    test_dataset=StrokeRecoveryDataset([folder / "test_online_coords.json"],
                            img_height = 60,
                            num_of_channels = 1.,
                            max_images_to_load = test_size,
                            root=r"../../data",
                            )

    test_dataloader = DataLoader(test_dataset,
                                  batch_size=batch_size,
                                  shuffle=True,
                                  num_workers=3,
                                  collate_fn=test_dataset.collate,
                                  pin_memory=False)

    device="cuda"
    globals().update(locals())
    return test_dataloader

def get_instance():
#     if not 'test_dataloader' in globals():
#         test_dataloader = loader()
    example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
    instance = example["gt_list"][0].numpy()
    return instance, example

def test_drawing():
    instance, example = get_instance()
    
    draw_from_gt(instance)
    draw_from_raw(gt_to_raw(instance))

loader()
test_drawing()

example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
instance = example["gt_list"][0].numpy()

/media/data/GitHub/simple_hwr/hwr_utils/testing
/media/data/GitHub/simple_hwr/hwr_utils/testing
../../data/online_coordinate_data/MAX_stroke_vTEST_AUGMENTFull/test_online_coords.json


INFO:root.hwr_utils.stroke_dataset:('Original dataloader figsize', 501)
INFO:root.hwr_utils.stroke_dataset:('Dataloader figsize', 5)
100%|██████████| 5/5 [00:00<00:00, 927.74it/s]


In [ ]:
from time import sleep
device="cuda"
example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
output = Path("./TEST_GRAPHING")
print(example.keys())

# This graphs the directions etc.
# graph(example, save_folder=output, x_relative_positions=x_relative_positions)

def draw_temp(stroke_list):
    fd, path = tempfile.mkstemp()
    path = "./temp"
    path += ".png"
    ratio = None
    try:
        draw_strokes(stroke_list, ratio, save_path=path, line_width=.8)
        img = read_img(path)
    finally:
        #os.remove(path)
        pass
    return img


In [ ]:
example = next(iter(test_dataloader)) # BATCH, WIDTH, VOCAB
instance = example["gt_list"][0].numpy()
start_points = stroke_recovery.relativefy(instance[:,2])
s = start_points
start_indices = np.argwhere(s==1).astype(int).reshape(-1)
l = np.split(instance[:,0:2], start_indices)

output = []

for item in l:
    if item.shape[0]:
        output.append({"x": item[:,0].tolist(), "y": item[:,1].tolist()})

#print(output)
draw_strokes(output, x_to_y=None)

img = draw_temp(output)
print(img.shape)

In [ ]:
path = "./temp"
img = read_img(path)


In [ ]:
import dill
dill.dump_session('notebook_env.db')
#dill.load_session('notebook_env.db')

print(output)

In [ ]:
print(example.keys())

In [ ]:
# Test draw
img  = Image.new("L", (50, 60), 255)
draw = ImageDraw.Draw(img)
draw.line([(0,0), (50,50)])
img.show()

In [ ]:
warp_points


In [ ]:
x = np.random.randint(5,10,(10,10))

In [ ]:
top = 1
bottom = 4
left = 2
right = 3

np.pad(x, ((top,bottom),(left,right)))


In [ ]:
y = x[0]+1

In [ ]:
print(y)

In [ ]:
print(x)

In [ ]:
instance, example = get_instance()

print(instance.shape)
img = draw_from_gt(instance, show=False)
new_gt, new_img = random_pad(instance, img, 20, 20)

img = Image.fromarray(new_img, 'L')
img.show()
draw_from_gt(new_gt, show=True)


In [ ]:
#loader()
from stroke_plotting import *
instance, example = get_instance()

print(instance.shape)
get_x_y_min_max_from_gt(instance)
img = draw_from_gt(instance, show=True)
new_gt = random_pad(instance, 15, 15)
draw_from_gt(new_gt, show=True, right_padding="random")


In [ ]:
img = draw_from_gt(instance, show=True)
img = blur(img.copy() / 127.5 - 1.0, max_intensity=1.1)
print(img)
img = (img + 1.0) * 127.5
f = Image.fromarray(np.uint8(img), 'L')
f.show()



In [ ]:
from hwr_utils.grid_distortion import *
img = draw_from_gt(instance, show=True)
img = Image.fromarray(img, 'L')
img.show()

if False:
    img = warp_points(instance, random_state=None)
    img = np.c_[img, instance[:,2]]


In [ ]:
## DISTORT IMAGE
from hwr_utils.grid_distortion  import *
img_prime = draw_from_gt(instance, show=False, max_width=3)
for i in range(0,1):
    img = img_prime.copy()
    if True:
        img = gaussian_noise(
                blur(
                    random_distortions(img_prime.copy()),
                    max_intensity=1.3),
                max_intensity=.1)
    f = Image.fromarray(np.uint8(img), 'L')
    #f.show()
# Render GT on top of image
f = f.convert("RGB")
img_prime = draw_from_gt(instance, show=False, linewidth=1, color=[255,0,0], alpha=True)
gt = Image.fromarray(np.uint8(img_prime), 'RGBA')
f.paste(gt, (0, 0), gt)
f.show()

#f = Image.fromarray(np.uint8(img), 'L')
    


In [ ]:
from matplotlib import pyplot as plt
l = []
max_width = 3
for i in range(1000):
    l.append(max(int(abs(np.random.randn())*(max_width-1)*.5 + 1 ),1))

plt.hist(l)

In [ ]:
### WARP

for i in range(10):
    gt = instance.copy()
    height = 61
    x = warp_points(gt*height)/height # convert to pixel space
    x = np.c_[x, instance[:,2]]
    img = draw_from_gt(x, show=True, max_width=2, save_path="./warp.png")


In [ ]:
m = np.array([[0,2,3,4,5],[0,2,3,4,5]])
m = m.reshape(-1,2)
print(m.shape)
x = warp_points(m)
print(x)

In [ ]:
f.show()



In [ ]:
im = Image.fromarray(np.uint8(img), 'L')
transparent_area = (0,0,50,50)

mask=Image.new('L', im.size, color=255)
draw=ImageDraw.Draw(mask) 
draw.rectangle(transparent_area, fill=0)
im.putalpha(mask)
im.show()

In [43]:
def show(numpy_array):
    f = Image.fromarray(np.uint8(numpy_array), 'L')
    f.show()
    return f

img_prime = draw_from_gt(instance, show=False, max_width=3)
background = Image.fromarray(np.uint8(img_prime), 'L')

for i in range(0,10):
    img = img_prime.copy()

    if False:
        img = random_distortions(img_prime.copy(), sigma=20.0, noise_max=2)
    else:
        img = blur(img_prime.copy(), max_intensity=1.0)
    
    show(img)
    # OVERLAY
    #bg = overlay_images(background_img=img, foreground_gt=instance, normalized=False)
    #bg.show()
